In [ ]:

# ==========================================
# Monte Carlo Simulation & Linear Regression
# Stock Price Analysis Tool
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from datetime import timedelta, datetime
from google.colab import files
from matplotlib.backends.backend_pdf import PdfPages
import os

# ==========================================
# CONFIGURATION
# ==========================================

# Dark theme configuration
plt.style.use('dark_background')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['figure.facecolor'] = '#0d1117'
plt.rcParams['axes.facecolor'] = '#161b22'
plt.rcParams['axes.edgecolor'] = '#30363d'
plt.rcParams['axes.labelcolor'] = '#c9d1d9'
plt.rcParams['xtick.color'] = '#8b949e'
plt.rcParams['ytick.color'] = '#8b949e'
plt.rcParams['text.color'] = '#c9d1d9'
plt.rcParams['grid.color'] = '#30363d'
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['grid.linewidth'] = 0.8
plt.rcParams['font.size'] = 11

# Enable minor grid
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.which'] = 'both'

# Color scheme
COLORS = {
    'primary': '#58a6ff',
    'secondary': '#f78166',
    'accent': '#56d364',
    'warning': '#f0883e',
    'purple': '#bc8cff',
    'historical': '#58a6ff',
    'simulation': '#f78166',
    'mean': '#56d364',
    'regression': '#f78166',
    'band1': '#f0883e',
    'band2': '#bc8cff'
}

# Report storage
REPORT_DATA = {
    'figures': [],
    'stats': [],
    'file_name': '',
    'analysis_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# ==========================================
# DATA LOADING & PREPARATION
# ==========================================

def load_and_prepare_data():
    """
    Load and prepare financial data from CSV or Excel file.
    Returns cleaned DataFrame with calculated log returns.
    """
    print("📊 Upload your CSV or Excel file...")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]

    # Load file
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name, parse_dates=['Date'])
    elif file_name.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_name, parse_dates=['Date'])
    else:
        raise ValueError("Unsupported file format. Please upload CSV or Excel.")

    # Validate and prepare data
    if 'Date' not in df.columns:
        raise ValueError("File must contain a 'Date' column.")

    df = df.sort_values(by='Date')
    df.set_index('Date', inplace=True)

    # Handle French/English column names
    if 'Clôture' in df.columns and 'Close' not in df.columns:
        df.rename(columns={'Clôture': 'Close'}, inplace=True)

    if 'Close' not in df.columns:
        raise ValueError("File must contain a 'Close' or 'Clôture' column.")

    # Clean price data
    df['Close'] = (df['Close'].astype(str)
                   .str.replace('€', '').str.replace('$', '')
                   .str.replace(' ', '').str.replace(',', '.')
                   .astype(float))

    # Calculate log returns
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df.dropna(inplace=True)

    if df.empty:
        raise ValueError("No valid data after processing.")

    print(f"✅ Loaded: {file_name}")
    print(f"📈 Data points: {len(df)}")
    print(f"📅 Period: {df.index.min().date()} to {df.index.max().date()}\n")

    return df, file_name

# ==========================================
# MONTE CARLO SIMULATION
# ==========================================

def monte_carlo_simulation(df, file_name):
    """
    Perform Monte Carlo simulation for stock price forecasting.
    Generates probabilistic price paths using geometric Brownian motion.
    """
    # Calculate parameters
    mu = df['Log_Return'].mean()
    sigma = df['Log_Return'].std()
    last_price = df['Close'].iloc[-1]
    num_simulations = 1000
    trading_days_per_year = 252

    # Forecast horizons
    forecast_periods = {
        '5 Years': 5 * trading_days_per_year,
        '10 Years': 10 * trading_days_per_year
    }

    print("🎲 MONTE CARLO SIMULATION")
    print(f"{'='*50}")
    print(f"Drift (μ): {mu:.6f}")
    print(f"Volatility (σ): {sigma:.6f}")
    print(f"Current Price: ${last_price:.2f}")
    print(f"Simulations: {num_simulations:,}\n")

    for period_name, num_days in forecast_periods.items():
        print(f"⏱️  Running {period_name} simulation...")

        # Initialize price paths
        price_paths = np.zeros((num_days, num_simulations))
        price_paths[0] = last_price

        # Run simulations
        for s in range(num_simulations):
            daily_returns = np.random.normal(mu, sigma, num_days - 1)
            price_multipliers = np.exp(daily_returns)
            for t in range(1, num_days):
                price_paths[t, s] = price_paths[t-1, s] * price_multipliers[t-1]

        # Generate forecast dates
        last_date = df.index[-1]
        forecast_dates = [last_date + timedelta(days=i) for i in range(1, num_days + 1)]

        # Calculate statistics
        final_prices = price_paths[-1, :]
        mean_path = np.mean(price_paths, axis=1)
        percentile_5 = np.percentile(price_paths, 5, axis=1)
        percentile_95 = np.percentile(price_paths, 95, axis=1)

        # Plotting
        fig, ax = plt.subplots(figsize=(18, 10))

        # Historical data
        ax.plot(df.index, df['Close'], color=COLORS['historical'],
                linewidth=2.5, label='Historical Price', zorder=3)

        # Individual simulations (sample)
        sample_size = min(100, num_simulations)
        for s in range(sample_size):
            ax.plot(forecast_dates, price_paths[:, s],
                   color=COLORS['simulation'], alpha=0.02, linewidth=0.5)

        # Mean path and confidence intervals
        ax.plot(forecast_dates, mean_path, color=COLORS['mean'],
                linewidth=3, label=f'Expected Path', zorder=4, linestyle='--')
        ax.fill_between(forecast_dates, percentile_5, percentile_95,
                        color=COLORS['accent'], alpha=0.15,
                        label='90% Confidence Interval', zorder=2)

        # Styling
        ax.set_title(f'Monte Carlo Simulation - {file_name.replace(".csv", "").replace(".xlsx", "")} ({period_name})',
                    fontsize=18, fontweight='bold', pad=20, color=COLORS['primary'])
        ax.set_xlabel('Date', fontsize=14, fontweight='bold')
        ax.set_ylabel('Price ($)', fontsize=14, fontweight='bold')
        ax.legend(loc='upper left', fontsize=12, framealpha=0.9)

        # Enhanced grid
        ax.grid(True, which='major', alpha=0.5, linewidth=0.8, linestyle='--')
        ax.grid(True, which='minor', alpha=0.2, linewidth=0.5, linestyle=':')
        ax.minorticks_on()

        # Add vertical line at forecast start
        ax.axvline(x=last_date, color=COLORS['warning'],
                  linestyle=':', linewidth=2, alpha=0.6, label='Forecast Start')

        plt.tight_layout()

        # Save figure for report
        REPORT_DATA['figures'].append(fig)

        plt.show()

        # Print statistics
        stats_text = f"\n📊 {period_name} Forecast Statistics:\n"
        stats_text += f"{'─'*50}\n"
        stats_text += f"Expected Final Price: ${np.mean(final_prices):.2f}\n"
        stats_text += f"Standard Deviation: ${np.std(final_prices):.2f}\n"
        stats_text += f"95% Confidence Interval: [${np.percentile(final_prices, 2.5):.2f}, ${np.percentile(final_prices, 97.5):.2f}]\n"
        stats_text += f"Potential Return: {((np.mean(final_prices) / last_price - 1) * 100):.2f}%\n"

        print(stats_text)
        REPORT_DATA['stats'].append({
            'type': f'Monte Carlo - {period_name}',
            'text': stats_text
        })

# ==========================================
# LINEAR REGRESSION WITH BANDS
# ==========================================

def linear_regression_with_bands(df, file_name):
    """
    Perform linear regression analysis with confidence bands.
    Shows price trend and deviation patterns.
    """
    print("📈 LINEAR REGRESSION ANALYSIS")
    print(f"{'='*50}")

    df_reg = df.copy()
    df_reg['Days_Since_Start'] = (df_reg.index - df_reg.index.min()).days

    # Fit model
    X = df_reg['Days_Since_Start'].values.reshape(-1, 1)
    y = df_reg['Close'].values

    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    residuals = y - predictions
    std_dev = np.std(residuals)

    print(f"Daily Trend: ${model.coef_[0]:.4f}/day")
    print(f"Annualized Trend: ${model.coef_[0] * 252:.2f}/year")
    print(f"Residual Std Dev: ${std_dev:.2f}\n")

    # Calculate bands
    upper_1std = predictions + 1 * std_dev
    lower_1std = predictions - 1 * std_dev
    upper_2std = predictions + 2 * std_dev
    lower_2std = predictions - 2 * std_dev

    # Plotting
    fig, ax = plt.subplots(figsize=(18, 10))

    # Price data
    ax.plot(df_reg.index, df_reg['Close'], color=COLORS['historical'],
            linewidth=2, label='Actual Price', zorder=3)

    # Regression line
    ax.plot(df_reg.index, predictions, color=COLORS['regression'],
            linewidth=2.5, linestyle='--', label='Linear Trend', zorder=4)

    # ±2 std bands
    ax.fill_between(df_reg.index, lower_2std, upper_2std,
                    color=COLORS['band2'], alpha=0.15,
                    label='±2σ Band (95%)', zorder=1)
    ax.plot(df_reg.index, upper_2std, color=COLORS['band2'],
            linewidth=1, linestyle=':', alpha=0.6)
    ax.plot(df_reg.index, lower_2std, color=COLORS['band2'],
            linewidth=1, linestyle=':', alpha=0.6)

    # ±1 std bands
    ax.fill_between(df_reg.index, lower_1std, upper_1std,
                    color=COLORS['band1'], alpha=0.25,
                    label='±1σ Band (68%)', zorder=2)
    ax.plot(df_reg.index, upper_1std, color=COLORS['band1'],
            linewidth=1.5, linestyle=':', alpha=0.8)
    ax.plot(df_reg.index, lower_1std, color=COLORS['band1'],
            linewidth=1.5, linestyle=':', alpha=0.8)

    # Styling
    ticker = file_name.replace('.csv', '').replace('.xlsx', '').upper()
    ax.set_title(f'Linear Regression Analysis - {ticker}',
                fontsize=18, fontweight='bold', pad=20, color=COLORS['primary'])
    ax.set_xlabel('Date', fontsize=14, fontweight='bold')
    ax.set_ylabel('Price ($)', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left', fontsize=12, framealpha=0.9)

    # Enhanced grid
    ax.grid(True, which='major', alpha=0.5, linewidth=0.8, linestyle='--')
    ax.grid(True, which='minor', alpha=0.2, linewidth=0.5, linestyle=':')
    ax.minorticks_on()

    plt.tight_layout()

    # Save figure and stats for report
    REPORT_DATA['figures'].append(fig)

    stats_text = f"\n📈 Linear Regression Statistics:\n"
    stats_text += f"{'─'*50}\n"
    stats_text += f"Daily Trend: ${model.coef_[0]:.4f}/day\n"
    stats_text += f"Annualized Trend: ${model.coef_[0] * 252:.2f}/year\n"
    stats_text += f"Residual Std Dev: ${std_dev:.2f}\n"

    REPORT_DATA['stats'].append({
        'type': 'Linear Regression',
        'text': stats_text
    })

    plt.show()

# ==========================================
# REPORT GENERATION
# ==========================================

def generate_report(df):
    """
    Generate a comprehensive PDF report with all analysis results.
    Automatically downloads the report file.
    """
    print("\n📄 Generating PDF Report...")

    # Create output filename
    ticker = REPORT_DATA['file_name'].replace('.csv', '').replace('.xlsx', '').upper()
    output_filename = f"Financial_Analysis_Report_{ticker}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"

    # Create PDF
    with PdfPages(output_filename) as pdf:
        # Cover page
        fig = plt.figure(figsize=(11, 8.5))
        fig.patch.set_facecolor('#0d1117')
        ax = fig.add_subplot(111)
        ax.axis('off')

        # Title
        title_text = f"FINANCIAL ANALYSIS REPORT\n\n{ticker}"
        ax.text(0.5, 0.7, title_text,
                ha='center', va='center',
                fontsize=32, fontweight='bold',
                color=COLORS['primary'])

        # Metadata
        metadata = f"Analysis Date: {REPORT_DATA['analysis_date']}\n"
        metadata += f"Data Points: {len(df):,}\n"
        metadata += f"Period: {df.index.min().date()} to {df.index.max().date()}\n"
        metadata += f"Current Price: ${df['Close'].iloc[-1]:.2f}"

        ax.text(0.5, 0.4, metadata,
                ha='center', va='center',
                fontsize=14,
                color='#c9d1d9',
                bbox=dict(boxstyle='round,pad=1',
                         facecolor='#161b22',
                         edgecolor=COLORS['primary'],
                         linewidth=2))

        # Footer
        ax.text(0.5, 0.1, "Monte Carlo Simulation & Linear Regression Analysis",
                ha='center', va='center',
                fontsize=12, style='italic',
                color='#8b949e')

        pdf.savefig(fig, facecolor='#0d1117')
        plt.close()

        # Statistics page
        fig = plt.figure(figsize=(11, 8.5))
        fig.patch.set_facecolor('#0d1117')
        ax = fig.add_subplot(111)
        ax.axis('off')

        # Title
        ax.text(0.5, 0.95, "ANALYSIS SUMMARY",
                ha='center', va='top',
                fontsize=24, fontweight='bold',
                color=COLORS['primary'])

        # Statistics
        y_position = 0.85
        for stat in REPORT_DATA['stats']:
            ax.text(0.1, y_position, stat['text'],
                    ha='left', va='top',
                    fontsize=11,
                    color='#c9d1d9',
                    family='monospace',
                    bbox=dict(boxstyle='round,pad=0.8',
                             facecolor='#161b22',
                             edgecolor='#30363d',
                             linewidth=1))
            y_position -= 0.25

        pdf.savefig(fig, facecolor='#0d1117')
        plt.close()

        # Add all figures
        for fig in REPORT_DATA['figures']:
            pdf.savefig(fig, facecolor='#0d1117')

        # Set PDF metadata
        d = pdf.infodict()
        d['Title'] = f'Financial Analysis Report - {ticker}'
        d['Author'] = 'Financial Analysis Tool'
        d['Subject'] = 'Monte Carlo Simulation & Linear Regression'
        d['Keywords'] = 'Finance, Monte Carlo, Linear Regression, Stock Analysis'
        d['CreationDate'] = datetime.now()

    print(f"✅ Report generated: {output_filename}")

    # Download file
    print("📥 Downloading report...")
    files.download(output_filename)
    print("✅ Download complete!\n")

    return output_filename

# ==========================================
# MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    print("\n" + "="*60)
    print("  FINANCIAL ANALYSIS TOOL")
    print("  Monte Carlo Simulation & Linear Regression")
    print("="*60 + "\n")

    try:
        # Load data
        df, file_name = load_and_prepare_data()
        REPORT_DATA['file_name'] = file_name

        # Run analyses
        monte_carlo_simulation(df, file_name)
        linear_regression_with_bands(df, file_name)

        # Generate and download report
        report_file = generate_report(df)

        print("\n" + "="*60)
        print("✅ Analysis completed successfully!")
        print(f"📄 Report saved: {report_file}")
        print("="*60 + "\n")

    except Exception as e:
        print(f"\n❌ Error: {e}\n")
